In [ ]:
%matplotlib widget
%config InlineBackend.figure_format = 'svg'
# %load_ext line_profiler
# Example: %lprun -T lprof0 -f celeri.celeri.get_okada_displacements u_east, u_north, u_up = celeri.get_okada_displacements(segment.lon1.values[segment_idx], segment.lat1[segment_idx], segment.lon2[segment_idx], segment.lat2[segment_idx], segment.locking_depth[segment_idx], segment.burial_depth[segment_idx], segment.dip[segment_idx], command.material_lambda, command.material_mu, 1, 0, 0, station.lon, station.lat)

In [ ]:
import addict
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from importlib import reload
from tqdm.notebook import tqdm

import celeri
celeri = reload(celeri)

# Plotting the global model is much much faster with tex fonts turned off
plt.rcParams['text.usetex'] = False

In [ ]:
command_file_name = "./data/western_north_america/basic_command.json"
command, segment, block, meshes, station, mogi, sar = celeri.read_data(command_file_name)
station = celeri.process_station(station, command)
segment = celeri.process_segment(segment, command, meshes)
sar = celeri.process_sar(sar, command)
closure, block = celeri.assign_block_labels(segment, station, block, mogi, sar)
# celeri.plot_block_labels(segment, block, station, closure)

In [ ]:
assembly = addict.Dict()
operators = addict.Dict()
assembly = celeri.merge_geodetic_data(assembly, station, sar) # Not sure this work correctly

# Build all linear operators
operators.block_rotation = celeri.get_block_rotation_operator(station)
operators.global_float_block_rotation = celeri.get_global_float_block_rotation_operator(station)
assembly, operators.block_motion_constraints = celeri.block_constraints(assembly, block, command)
assembly, operators.slip_rate_constraints = celeri.slip_rate_constraints(assembly, segment, block, command)
operators.slip_rate_segment_block = celeri.get_fault_slip_rate_partials(segment, block)
operators.block_strain_rate, strain_rate_block_idx = celeri.get_strain_rate_centroid_operator(block, station, segment)
operators.mogi_station = celeri.get_mogi_operator(mogi, station, command)

# Elastic operators
operators.okada_segment_station = celeri.get_segment_station_operator_okada(segment, station, command)
# celeri.plot_segment_displacements(segment, station, command, segment_idx=0, strike_slip=1, dip_slip=0, tensile_slip=0, lon_min=235, lon_max=255, lat_min=30, lat_max=50, quiver_scale=1e-1)
operators.tri_station = celeri.get_tri_station_operator_okada(meshes, station, command)

# Get additional matrix shape information for assembly
assembly.index.sz_elastic = operators.okada_segment_station.shape # Not sure this is correct
assembly.index.sz_slip = operators.slip_rate_segment_block.shape # Not sure this is correct
assembly.index.sz_rotation = operators.block_rotation.shape # Not sure this is correct
assembly = celeri.station_row_keep(assembly) # Not sure this is correct

In [ ]:
celeri.get_all_mesh_smoothing_matrices(meshes)  #TODO: I don't think this is right because I only see terms on the main diagonal.
plt.figure()
plt.imshow(meshes[0].smoothing_matrix.toarray()[0:40, 0:40])
plt.colorbar()
plt.show()

In [ ]:
import copy
# Let's prescribe and recover a Cascadia slip model without any blocks.  Essentially coseismic slip

def get_synthetic_displacements(mesh, tri_linear_operator):
    tri_centroid_to_mesh_lon = mesh.centroids[:, 0] - np.mean(mesh.centroids[:, 0])
    tri_centroid_to_mesh_lat = mesh.centroids[:, 1] - np.mean(mesh.centroids[:, 1])
    tri_centroid_to_mesh_centroid_distance = np.sqrt(tri_centroid_to_mesh_lon ** 2 + tri_centroid_to_mesh_lat ** 2)
    dip_slip_distribution = np.exp(-(tri_centroid_to_mesh_centroid_distance / 1.0) ** 2.0)
    slip_distribution = np.zeros(2 * dip_slip_distribution.size)
    slip_distribution[1::2] = dip_slip_distribution # Dip slip only
    synthetic_displacements = tri_linear_operator @ slip_distribution
    return slip_distribution, synthetic_displacements


# Shrink operators.tri_station so that there are no vertical displacments and no tensile slip
T = copy.deepcopy(operators.tri_station)
T = np.delete(T, np.arange(2, T.shape[0], 3), axis=0)
T = np.delete(T, np.arange(2, T.shape[1], 3), axis=1)

# Generate Guassian slip source and synthetic displacements
slip_distribution, synthetic_displacements = get_synthetic_displacements(meshes[0], T)

# Visualize prescibed slip and synthetic displacements
plt.figure()
plt.scatter(meshes[0].centroids[:, 0], meshes[0].centroids[:, 1], s=2, c=slip_distribution[1::2], alpha=0.5)
plt.colorbar()
plt.quiver(station.lon, station.lat, synthetic_displacements[0::2], synthetic_displacements[1::2], scale=2e0)
plt.gca().set_aspect("equal", adjustable="box")
plt.xlim([230.0, 245.0])
plt.ylim([37.5, 52.5])
plt.show()

# TODO: Arrange linear system for solve

# TODO: Solve for coseismic slip distribution
estimated_slip_distribution = np.linalg.lstsq(T, synthetic_displacements, rcond=None)


# TODO: Visualized prescribed and recovered slip
plt.figure()
plt.scatter(meshes[0].centroids[:, 0], meshes[0].centroids[:, 1], s=2, c=estimated_slip_distribution[0][1::2], alpha=0.5)
plt.colorbar()
# plt.quiver(station.lon, station.lat, synthetic_displacements[0::2], synthetic_displacements[1::2], scale=2e0)
plt.gca().set_aspect("equal", adjustable="box")
plt.xlim([230.0, 245.0])
plt.ylim([37.5, 52.5])
plt.show()

In [ ]:
plt.figure(figsize=(12, 7))

plt.subplot(2, 2, 1)
plt.title("input strike-slip")
plt.scatter(meshes[0].centroids[:, 0], meshes[0].centroids[:, 1], s=2, c=slip_distribution[0::2], alpha=0.5)
plt.colorbar()
plt.gca().set_aspect("equal", adjustable="box")
plt.xlim([230.0, 245.0])
plt.ylim([37.5, 52.5])

plt.subplot(2, 2, 2)
plt.title("input dip-slip")
plt.scatter(meshes[0].centroids[:, 0], meshes[0].centroids[:, 1], s=2, c=slip_distribution[1::2], alpha=0.5)
plt.colorbar()
plt.gca().set_aspect("equal", adjustable="box")
plt.xlim([230.0, 245.0])
plt.ylim([37.5, 52.5])

plt.subplot(2, 2, 3)
plt.title("estimated strike-slip")
plt.scatter(meshes[0].centroids[:, 0], meshes[0].centroids[:, 1], s=2, c=estimated_slip_distribution[0][0::2], alpha=0.5)
plt.colorbar()
plt.gca().set_aspect("equal", adjustable="box")
plt.xlim([230.0, 245.0])
plt.ylim([37.5, 52.5])

plt.subplot(2, 2, 4)
plt.title("estimated dip-slip")
plt.scatter(meshes[0].centroids[:, 0], meshes[0].centroids[:, 1], s=2, c=estimated_slip_distribution[0][1::2], alpha=0.5)
plt.colorbar()
plt.gca().set_aspect("equal", adjustable="box")
plt.xlim([230.0, 245.0])
plt.ylim([37.5, 52.5])
plt.show()

In [ ]:
estimated_slip_distribution[0]